# Home Cage Dopamine Analysis

In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_ROOT)

from trial_class import *
from experiment_class import Experiment

from hc_extension import *
from Hab_Dishab.hd_extension import get_trial_dataframes, create_metadata_dataframe, create_da_metrics_dataframe, plot_peak_for_subsequent_investigations_custom

ImportError: cannot import name 'create_metadata_dataframe' from 'Hab_Dishab.hd_extension' (/Users/naylajimenez/Documents/GitHub/Fiber_Photometry/Hab_Dishab/hd_extension.py)

In [ ]:
# experiment_path = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined\Home_Cage\nac"
# csv_base_path = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined\Home_Cage\nac_csvs"
# brain_region = '#15616F'

# NAc: #15616F
# mPFC: #FFAF00

experiment_path = r"/Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Home_Cage/all/nac"
csv_base_path = r"//Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Home_Cage/all_csvs/nac_csvs"
brain_region = '#15616F'

In [ ]:
# groups csv + experiment data into one variable
experiment = Experiment(experiment_path, csv_base_path)

# batch process the data, removing the specified time segments for subjects
experiment.default_batch_process()

In [ ]:
bout_definitions = [
    {'prefix': 'Short_Term', 'introduced': 'Short_Term_Introduced', 'removed': 'Short_Term_Removed'},
    {'prefix': 'Long_Term', 'introduced': 'Long_Term_Introduced', 'removed': 'Long_Term_Removed'},
    {'prefix': 'Novel', 'introduced': 'Novel_Introduced', 'removed': 'Novel_Removed'}
]

experiment.group_extract_manual_annotations(bout_definitions, first_only=True)

### Peak standard z-score

In [ ]:
# total_avg_bout_duration = metadata_df["Average Bout Duration"].mean()
# print(f"Total Average Bout Duration: {total_avg_bout_duration:.4f}")
# Proceed with DA metric computation after all files are processed
experiment.compute_all_da_metrics(use_max_length=False,
                                  max_bout_duration=5, #otal_avg_bout_duration
                                  use_adaptive=False, 
                                  allow_bout_extension=False,
                                  mode='standard')

In [ ]:
exp_da_dict = get_trial_dataframes(experiment)

In [ ]:
desired_bouts = ['Short_Term-1', 'Novel-1', 'Short_Term-2', 'Long_Term-1']
da_metadata_df = create_da_metrics_dataframe(exp_da_dict, behavior="Investigation", desired_bouts=desired_bouts)

In [ ]:
da_metadata_df

In [ ]:
new_da_df = da_metadata_df.copy() 

# Desired bout order
desired_bout_order = ["Short_Term-1", "Short_Term-2", "Long_Term-1", "Novel-1"]
clean_labels = ["Acq-ST", "Short Term", "Long Term", "Novel"]
xtick_colors = ["teal", "blue", "purple", "orange"]

# Map the bout column to a categorical with desired order
df = new_da_df[new_da_df["Bout"].isin(desired_bout_order)].copy()
df["Bout"] = pd.Categorical(df["Bout"], categories=desired_bout_order, ordered=True)

In [ ]:
from scipy.stats import ttest_rel
import numpy as np
import matplotlib.pyplot as plt

def dopamine(precomputed_df, 
             metric_name="Mean Z-score", 
             title="Combined DA Metrics", 
             ylabel="DA Metric", 
             xlabel="Bout", 
             custom_xtick_labels=None, 
             custom_xtick_colors=None, 
             ylim=None, 
             bar_color="#00B7D7", 
             yticks_increment=None, 
             figsize=(14,8), 
             pad_inches=0.1,
             save=False,
             save_name=None):
    """
    Plots DA metrics across specific bouts using a provided DataFrame.
    Prints paired t-test results comparing first bout to others.
    """
    def perform_t_tests(pivot_df):
        results = {}
        bout_names = pivot_df.columns.tolist()
        for i in range(1, len(bout_names)):
            bout1, bout2 = bout_names[0], bout_names[i]
            paired_df = pivot_df[[bout1, bout2]].dropna()
            if len(paired_df) > 1:
                t_stat, p_value = ttest_rel(paired_df[bout1], paired_df[bout2])
                significance = "ns"
                if p_value < 0.001:
                    significance = "***"
                elif p_value < 0.01:
                    significance = "**"
                elif p_value < 0.05:
                    significance = "*"
                results[f"{bout1} vs {bout2}"] = {"t_stat": t_stat, "p_value": p_value, "significance": significance}
        return results

    df = precomputed_df.copy()

    try:
        pivot_df = df.pivot(index="Subject", columns="Bout", values=metric_name)
    except Exception as e:
        print("Error pivoting data:", e)
        return

    overall_stats = df.groupby("Bout")[metric_name].agg(['mean', 'sem']).reset_index()
    t_test_results = perform_t_tests(pivot_df)

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=figsize)
    bar_positions = np.arange(len(overall_stats))

    ax.bar(bar_positions, overall_stats["mean"], yerr=overall_stats["sem"],
           capsize=6, color=bar_color, edgecolor='black', linewidth=5, width=0.6,
           error_kw=dict(elinewidth=3, capthick=3, zorder=5))

    for subject_id in pivot_df.index:
        ax.plot(bar_positions, pivot_df.loc[subject_id], linestyle='-', color='gray', 
                alpha=0.5, linewidth=3, marker='o', markerfacecolor='none', 
                markeredgecolor='gray', markeredgewidth=2, markersize=10)

    ax.set_ylabel(ylabel, fontsize=35, labelpad=12)
    ax.set_xlabel(xlabel, fontsize=35, labelpad=12)
    ax.set_title(title, fontsize=28)

    # X-ticks
    ax.set_xticks(bar_positions)
    xtick_labels = custom_xtick_labels if custom_xtick_labels else overall_stats["Bout"].tolist()
    ax.set_xticklabels(xtick_labels, fontsize=28)
    if custom_xtick_colors:
        for tick, color in zip(ax.get_xticklabels(), custom_xtick_colors):
            tick.set_color(color)

    ax.tick_params(axis='y', labelsize=35)
    ax.tick_params(axis='x', labelsize=35)

    if ylim:
        ax.set_ylim(ylim)
    else:
        all_vals = np.concatenate([pivot_df.values.flatten(), overall_stats["mean"].values])
        ax.set_ylim(0, np.nanmax(all_vals) * 1.2)

    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment))

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(5)
    ax.spines['bottom'].set_linewidth(5)

    plt.tight_layout()
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)
    plt.show()

    # --- Print T-Test Results ---
    if t_test_results:
        print("\nPaired t-test results (comparing first bout to others):")
        for comparison, stats in t_test_results.items():
            print(f"{comparison}: p = {stats['p_value']:.4f} ({stats['significance']})")


In [ ]:
dopamine(
    precomputed_df=df,
    metric_name="Max Peak",
    title=None,
    ylabel="Peak Z-scored ∆F/F",
    xlabel="Bout Type",
    custom_xtick_labels=clean_labels,
    custom_xtick_colors=xtick_colors,
    ylim=(-1, 9),
    yticks_increment=2,
    bar_color=brain_region,
    figsize=(14, 8),
    save=True,
    save_name ="NAc_C1-3"
)

## Subsequent Investigation

In [ ]:
experiment.reset_all_behaviors()
experiment.group_extract_manual_annotations(bout_definitions=bout_definitions, first_only = False)

# total_avg_bout_duration = metadata_df["Average Bout Duration"].mean()
# print(f"Total Average Bout Duration: {total_avg_bout_duration:.4f}")
# Proceed with DA metric computation after all files are processed
experiment.compute_all_da_metrics(use_max_length=False,
                                  max_bout_duration=4, #otal_avg_bout_duration
                                  use_adaptive=False, 
                                  allow_bout_extension=False,
                                  mode='standard')

In [ ]:
exp_da_dict = get_trial_dataframes(experiment)

In [ ]:
from scipy.optimize import curve_fit

def create_big_df_from_exp_da_dict(exp_da_dict):
    """
    Merges all subjects' DataFrames from exp_da_dict into one big DataFrame.
    Adds a 'Subject' column for each row.
    """
    all_list = []
    for subject_id, df_subj in exp_da_dict.items():
        df_copy = df_subj.copy()
        df_copy["Subject"] = subject_id
        all_list.append(df_copy)
    big_df = pd.concat(all_list, ignore_index=True)
    return big_df

def plot_peak_for_subsequent_investigations_custom(
    exp_da_dict,
    selected_bouts=None,             # e.g. ["s1-1","s1-2"]
    n_subsequent_investigations=3,   # e.g. keep first 3 investigations per (Subject, Bout)
    peak_col="Max Peak",             # which column holds the per-event peak DA
    metric_type='slope',             # choose 'slope' or 'decay'
    figsize=(14, 8),
    line_order=None,
    custom_colors=None,
    custom_legend_labels=None,
    custom_xtick_labels=None,
    ylim=None,
    ytick_increment=None,
    xlabel="Investigation Index",
    ylabel="Avg " + "Max Peak",
    plot_title="Average Peak per Investigation"
):
    """
    1) Merges all DataFrames in exp_da_dict into one big DataFrame.
    2) Filters for the specified bouts (e.g. ["s1-1", "s1-2"]).
    3) Within each (Subject, Bout), sorts by Event_Start and assigns an 'InvestigationIndex'
       (1 for the first event, 2 for the second, etc.).
    4) Keeps only the first n_subsequent_investigations per (Subject, Bout).
    5) Groups by (Bout, InvestigationIndex) across subjects and computes the average of peak_col.
    6) For each Bout, fits either a linear regression (if metric_type='slope') or an exponential decay
       (if metric_type='decay') to the AvgPeak vs. InvestigationIndex data.
       The computed value (slope or decay constant) is shown in the legend.
    7) Plots each Bout as a line using full custom visual styling.
    
    Returns the aggregated DataFrame used for plotting.
    """

    def exponential_decay(x, A, B, tau):
        """Basic exponential decay model: y = A + B * exp(-x / tau)"""
        return A + B * np.exp(-x / tau)

    # 1) Merge all subject data
    big_df = create_big_df_from_exp_da_dict(exp_da_dict)
    
    # 2) Filter for the chosen bouts if provided
    if selected_bouts is not None:
        big_df = big_df[big_df["Bout"].isin(selected_bouts)].copy()
    
    if big_df.empty:
        print("No data left after filtering for bouts. Nothing to plot.")
        return pd.DataFrame()
    
    # 3) Within each (Subject, Bout), sort by Event_Start and assign an InvestigationIndex
    big_df.sort_values(["Subject", "Bout", "Event_Start"], inplace=True)
    big_df["InvestigationIndex"] = big_df.groupby(["Subject", "Bout"]).cumcount() + 1
    
    # 4) Keep only the first n_subsequent_investigations per (Subject, Bout)
    big_df = big_df[big_df["InvestigationIndex"] <= n_subsequent_investigations]
    
    # 5) Group by (Bout, InvestigationIndex) and compute average peak and subject count
    agg_df = (
        big_df.groupby(["Bout", "InvestigationIndex"], as_index=False)
        .agg(
            SubjectCount=("Subject", "nunique"),
            AvgPeak=(peak_col, "mean")
        )
    )
    
    # 6) Create figure with custom styling
    if custom_colors is None:
        custom_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(5)
    ax.spines["bottom"].set_linewidth(5)
    ax.tick_params(axis="both", which="major", labelsize=48)
    
    metrics_dict = {}  # to store computed metric for each Bout
    
    # Determine unique bouts to plot (order by line_order if provided)
    if line_order is None:
        unique_bouts = sorted(agg_df["Bout"].unique())
    else:
        unique_bouts = line_order
    
    # 7) For each Bout, fit the data and plot the line
    for i, bout in enumerate(unique_bouts):
        df_line = agg_df[agg_df["Bout"] == bout].copy()
        df_line.sort_values("InvestigationIndex", inplace=True)
        
        x_vals = df_line["InvestigationIndex"].values
        y_vals = df_line["AvgPeak"].values
        
        if len(x_vals) == 0 or len(y_vals) == 0:
            print(f"Skipping bout '{bout}' due to no data.")
            continue
        
        if metric_type.lower() == 'slope':
            slope, intercept, r_val, p_val, std_err = linregress(x_vals, y_vals)
            metrics_dict[bout] = slope
            metric_label = f"slope: {slope:.3f}"
        elif metric_type.lower() == 'decay':
            p0 = (np.min(y_vals), np.max(y_vals)-np.min(y_vals), 1.0)  # initial guess: A, B, tau
            try:
                popt, _ = curve_fit(exponential_decay, x_vals, y_vals, p0=p0)
                tau = popt[2]
                metrics_dict[bout] = tau
                metric_label = f"decay: {tau:.3f}"
            except RuntimeError:
                metrics_dict[bout] = np.nan
                metric_label = "decay: N/A"
                print(f"Warning: exponential fit failed for bout '{bout}'.")
        else:
            raise ValueError("metric_type must be 'slope' or 'decay'.")
        
        # Prepare legend text; incorporate custom legend labels if provided
        if custom_legend_labels and i < len(custom_legend_labels):
            legend_text = custom_legend_labels[i]
        else:
            legend_text = bout
        # Append computed metric and subject count (n) to legend text
        legend_text += f" ({metric_label}, n={df_line['SubjectCount'].max()})"
        
        color = custom_colors[i % len(custom_colors)]
        ax.plot(
            x_vals, y_vals,
            marker='o', linestyle='-',
            color=color,
            linewidth=5, markersize=30,
            label=legend_text
        )
    
    # 8) Set axis labels and formatting
    ax.set_xlabel(xlabel, fontsize=35, labelpad=12)
    ax.set_ylabel(ylabel, fontsize=35, labelpad=12)
    
    if ylim is not None:
        ax.set_ylim(ylim)
        if ytick_increment is not None:
            y_ticks = np.arange(ylim[0], ylim[1] + ytick_increment, ytick_increment)
            ax.set_yticks(y_ticks)
            y_tick_labels = [f"{int(yt)}" if float(yt).is_integer() else f"{yt:.1f}" for yt in y_ticks]
            ax.set_yticklabels(y_tick_labels, fontsize=44)
    
    if custom_xtick_labels:
        ax.set_xticks(np.arange(1, len(custom_xtick_labels) + 1))
        ax.set_xticklabels(custom_xtick_labels, fontsize=44)
    else:
        unique_x = sorted(agg_df["InvestigationIndex"].unique())
        ax.set_xticks(unique_x)
        ax.set_xticklabels([str(x) for x in unique_x], fontsize=44)
    
    if plot_title:
        ax.set_title(plot_title, fontsize=20)
    
    ax.legend(fontsize=26)
    plt.tight_layout()
    plt.savefig("my_plot.png", transparent=True, dpi=300)
    plt.show()
    
    print(f"\n=== Computed Metric ({metric_type.upper()}): ===")
    for bout, val in metrics_dict.items():
        print(f"Bout: {bout}, {metric_type} = {val:.3f}")
    
    return agg_df



In [ ]:
new_da_df = da_metadata_df.copy() 

# Desired bout order
xtick_colors = ['#A839A4','#E06928','#00B7D7','#0045A6'] 

df_final = plot_peak_for_subsequent_investigations_custom(
    exp_da_dict,
    selected_bouts=['Short_Term-1', 'Novel-1', 'Short_Term-2', 'Long_Term-1'],
    n_subsequent_investigations=5,
    custom_colors=xtick_colors,
    custom_legend_labels=["Long Term", "Novel", "Acq-ST", "Short Term"],
    peak_col="Max Peak",
    metric_type='slope', 
    ylim=(0, 3),
    ylabel='Average Peak ∆F/F'
)


In [ ]:
df_final